# P3 — Subscription Churn & Retention : 01_eda

In [ ]:
import pandas as pd, numpy as np
from sqlalchemy import create_engine, text
import matplotlib.pyplot as plt

DB = 'sqlite:///../../da4.db'
TABLE = 'p3_churn_stage'

engine = create_engine(DB)
with engine.begin() as conn:
    df = pd.read_sql(text(f'SELECT * FROM {TABLE}'), conn)

print('Shape:', df.shape)
display(df.head())
display(df.describe(numeric_only=True).T)
print('\nNull % (top):')
print((df.isna().mean()*100).sort_values(ascending=False).head(15))


# --- Churn rate overall & by plan/tenure
overall = df['churned'].mean()
print('Overall churn %:', round(overall*100,2))
by_plan = df.groupby('plan', as_index=False)['churned'].mean()
by_ten  = df.groupby('tenure_bucket', as_index=False)['churned'].mean().sort_values('tenure_bucket')
plt.figure(); plt.bar(by_plan['plan'], by_plan['churned']*100)
plt.title('Churn % by Plan'); plt.tight_layout(); plt.show()
plt.figure(); plt.bar(by_ten['tenure_bucket'], by_ten['churned']*100)
plt.title('Churn % by Tenure Bucket'); plt.tight_layout(); plt.show()
by_plan, by_ten


# --- Simple cohort retention proxy (share not churned by cohort)
coh = (df.groupby('cohort_month', as_index=False)
         .agg(active=('churned', lambda s: (1 - s).sum()),
              size=('churned','size')))
coh['retention_pct'] = coh['active'] / coh['size'] * 100
coh.sort_values('cohort_month', inplace=True)
plt.figure(); plt.plot(coh['cohort_month'], coh['retention_pct'])
plt.xticks(rotation=90); plt.title('Retention % by Signup Cohort'); plt.tight_layout(); plt.show()
coh


# --- At-risk breakdown
risk = df.groupby(['plan','country'], as_index=False)['at_risk'].mean().sort_values('at_risk', ascending=False).head(10)
risk
